<!-- Autoheader begin -->
<hr/>
<div id="navtitle_3_1_py" style="text-align:center; font-size:16px">III.1 Population Inversion in a Two-Level-System using Krotov's Method</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_2_3_chiral.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="py_exercise_2_3_chiral.ipynb" style="font-size:13px">II.3 Parameter Optimization of Three-Wave Mixing in a Three-Level System</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_2_1_TLS.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="py_exercise_2_1_TLS.ipynb" style="font-size:13px">II.1 Population Inversion in a Two-Level-System using Parameter Optimization</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_2_lambda.ipynb">next notebook $\rightarrow$</a><br>
        <a href="py_exercise_3_2_lambda.ipynb" style="font-size:13px">III.2 Optimal Control for STIRAP</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Julia/jl_exercise_3_1_TLS.ipynb">👉 Julia version</a></div>

---
<!-- Autoheader end -->

# Population Inversion in a Two-Level-System using Krotov's Method

$\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid
#2\vphantom{#1}\right\rangle}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}$

This notebook is the first illustration of using a complete optimal control framework with
gradient-based methods. It considers the example of inverting the
population in a two-level system that we have already explored in 
[Exercise I.1](py_exercise_1_1_TLS.ipynb) and 
[Exercise II.1](py_exercise_2_1_TLS.ipynb). These previous notebooks were aimed at finding control solutions by
tuning a small number of analytical parameters for a simple pulse shape,
either by hand, or by using a gradient-free numerical optimization method, respectively.

In this notebook, you will learn how to use numerical optimization to find
control fields $E(t)$ which are not limited to a simple, predefined analytical shape.
Instead, we treat the value of $E(t)$ at every point $t$ of a finely sampled time grid as an independent
control parameter. This allows for a lot of control parameters (as many as
there are points on the time grid) and thus a lot of freedom to find optimal solutions. Gradient-free methods are only suitable for a small amount of control parameters and are thus not appropriate for such a task. Instead, we employ methods which take into account the *gradient* of the optimization functional with respect to the control parameters (the values $E(t)$). Two of the more established methods for gradient-based optimization of fields $E(t)$ (approximated as piecewise-constant on a fine time grid) are "GRadient Ascent Pulse Engineering" (GRAPE) and
Krotov's method.

You will learn here the use of the [`krotov` Python
package](https://github.com/qucontrol/krotov) to formulate and solve the
control problem.

In [ ]:
import krotov

Despite the fact that the two-level system can also directly be solved with simpler approaches, this example still serves as a nice playground to to show you the tools and general approach for a general control problem that cannot be solved by varying a few parameters only.

## Setup

First, we need to load some of the libraries needed throughout this notebook.

In [ ]:
import numpy as np  # package for numerical functions such as cos, sin, etc.
import qutip  # QUantum Toolbox In Python
import matplotlib  # package for plotting
import matplotlib.pylab as plt

# Some functions for easy access:
from numpy import pi, sqrt, exp, sin, cos

We are using some Bloch sphere visualizations in this tutorial that benefit
from being interactive, so we will activate an interactive backend for
`matplotlib`:

In [ ]:
%matplotlib widget

Note that this requires the `ipympl` package to be installed in the same
version both in the project environment and in the environment providing the
Jupyter application. If you are having trouble with the plots in this
notebook, delete the above cell, restart the kernel, and rerun the notebook.
The plots won't be interactive, but you willl still be able to follow along this
tutorial.


## Define the Hamiltonian

In the
following the Hamiltonian, guess field, and states are defined.

The Hamiltonian
$\op{H}_{0} = - \frac{\omega}{2} \op{\sigma}_{z}$
represents a
simple qubit with energy
level splitting $\omega$ in the canonical basis
$\{\ket{0},\ket{1}\}$. The control
field
$\epsilon(t)$ is assumed to couple via
the
Hamiltonian $\op{H}_{1}(t) =
\epsilon(t) \op{\sigma}_{x}$ to the qubit,
i.e., the control
field effectively
drives
transitions between the two basis states.

In [ ]:
def ham_and_states(omega=1.0, eps0=(lambda t, args: 1.0)):
    """Two-level-system Hamiltonian

    Args:
        omega (float): energy separation of the qubit levels
        eps0 (func): The driving field eps0(t, args)
    """
    H0 = -0.5 * omega * qutip.operators.sigmaz()
    H1 = qutip.operators.sigmax()

    psi0 = qutip.Qobj(np.array([1, 0]))  # State |0⟩
    psi1 = qutip.Qobj(np.array([0, 1]))  # State |1⟩

    return ([H0, [H1, eps0]], psi0, psi1)

In addition, we define a shape function $S(t)$ which takes care of
experimental limits such as the necessity of finite ramps
at the beginning and
end of the control field.

In [ ]:
def S(t):
    """Shape function for the initial and field update"""
    return krotov.shapes.flattop(
        t, t_start=0, t_stop=10, t_rise=0.5, t_fall=0.5, func="sinsq"
    )

This shape function will be used in two contexts: First, in the optimization with Krotov's method later on in this tutorial, it will shape the pulse update, ensuring that the boundary conditions are maintained in every iteration of the optimization.

Second, we will also use $S(t)$ to multiply the guess field `eps0` when calling `ham_and_states`:

In [ ]:
def shape_field(eps0):
    """Applies the shape function S(t) to the guess field"""
    eps0_shaped = lambda t, args: eps0(t, args) * S(t)
    return eps0_shaped

This allows playing around with different functions for `eps0` that may or may not have suitable boundary conditions.

## Simulate dynamics of the guess field

Before heading towards the optimization
procedure, we first simulate the
dynamics under the guess field
$\epsilon_{0}(t)$.

However, before we can propagate the state under the guess field, we need to define the time grid of the
dynamics. As an example, we define the
initial state to be at time $t=0$ and
consider a total propagation time of
$T=4$. The entire time grid is divided into
$n_{t}=80$ equidistant time steps (corresponding to 81 time grid points).

In [ ]:
tlist = np.linspace(0, 10, 81)

Naturally, we also have to define the guess pulse itself.

In [ ]:
def guess_pulse(t, args):
    A = .1
    σ = 2
    E = A * exp(-((t - 5) ** 2) / (2 * σ ** 2)) * cos(3*t)
    return E

In the Hamiltonian, we multiply `guess_pulse` with the shape $S(t)$ via `shape_field` as mentioned above:

In [ ]:
H, psi0, psi1 = ham_and_states(eps0=shape_field(guess_pulse))

Feel free to play around with `guess_pulse` and the construction of the
Hamiltonian.

The total field looks as follows:

In [ ]:
def plot_pulse(pulse, tlist):
    fig, ax = plt.subplots(figsize=(8, 5))
    if callable(pulse):
        pulse = np.array([pulse(t, args=None) for t in tlist])
    ax.plot(tlist, pulse)
    ax.set_xlabel("time")
    ax.set_ylabel("pulse amplitude")
    plt.show(fig)


plt.close("all")
plot_pulse(H[1][1], tlist)

Then, we solve the equation of motion for the initial state
$\ket{\Psi_{\init}}=\ket{0}$ and the Hamiltonian $\op{H}(t)$ generating its
evolution. To this end, we define the projectors $\op{P}_0 = \ket{0}\bra{0}$
and $\op{P}_1 = \ket{1}\bra{1}$ to compute their expectation values.
Afterwards, we plot the dynamics.

In [ ]:
proj0 = psi0 * psi0.dag()
proj1 = psi1 * psi1.dag()

guess_dynamics = qutip.mesolve(H, psi0, tlist, e_ops=[proj0, proj1])


def plot_population(result, ylim=None):
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(result.times, result.expect[0], label="0")
    ax.plot(result.times, result.expect[1], label="1")
    ax.legend()
    ax.set_xlabel("time")
    ax.set_ylabel("population")
    if ylim is not None:
        ax.set_ylim(ylim)
    plt.show(fig)

plt.close("all")
plot_population(guess_dynamics)

Note that there is a small amount of oscillation in the dynamics, which we can see by zooming in the dynamics of the $\ket{1}$ state

In [ ]:
plt.close("all")
plot_population(guess_dynamics, ylim=(0, 0.0016))

We can do the same again, but plot the trajectory on the Bloch sphere
instead. Try to understand how the `mesolve` routine is different compared to
the one for plotting the population.

In [ ]:
σ_x = qutip.sigmax()
σ_y = qutip.sigmay()
σ_z = qutip.sigmaz()

guess_dynamics = qutip.mesolve(H, psi0, tlist, e_ops=[σ_x, σ_y, σ_z])


def plot_bloch(result):
    b = qutip.Bloch(view=[-60, 30])
    exp_x = result.expect[0]
    exp_y = result.expect[1]
    exp_z = result.expect[2]
    b.point_color = plt.get_cmap("viridis_r")(tlist / tlist[-1])  # set nice colormap
    b.add_points([exp_x, exp_y, exp_z], "m")
    b.frame_alpha = 0.1
    b.make_sphere()
    plt.show()

plt.close("all")
plot_bloch(guess_dynamics)

## Define the optimization target

We want to optimize a simple state-to-state
transfer
from initial state $\ket{\Psi_{\init}} = \ket{0}$ to the target state
$\ket{\Psi_{\tgt}} = \ket{1}$, which we want to reach at final time $T$. Note
that we also have to pass the Hamiltonian $\op{H}(t)$ that determines the
dynamics of
the system.

From a mathematical perspective we optimize the guess field $\epsilon_{0}(t)$ such
that the intended state-to-state transfer $\ket{\Psi_{\init}} \rightarrow
\ket{\Psi_{\tgt}}$ is solved.
To this end, we
choose the functional to be $F = F_{ss}$ with
\begin{equation}
F_{ss}
=
\left|\Braket{\Psi(T)}{\Psi_{\tgt}}\right|^2
\end{equation}

with
$\ket{\Psi(T)}$ the
forward propagated state of $\ket{\Psi_{\init}}$. Maximizing $F_{ss}$ is equivalent to minimizing the infidelity which we employ as our optimization functional, i.e.

In [ ]:
? krotov.functionals.J_T_ss

The functional is evaluated based on the initial state forward-propagated under a specific time-dependent Hamiltonian $\Op{H}(t)$ and considers the overlap with the associated target state. The initial state, Hamiltonian, and target state are collected in an `Objective` object that will be passed to the `J_T_ss` function.

In [ ]:
objectives = [krotov.Objective(initial_state=psi0, target=psi1, H=H)]

The *result* of the propagation is available to `J_T_ss` as `fw_states_T`.

## Using Krotov's method

In an optimization with Krotov's method, we have the option of using $S(t)$ as the `update_shape` for
$\epsilon_0(t)$. Wherever $S(t)$ is zero, the optimization will not change the
value of the control from the original guess. In general, this shape function can be different from the one used to shape the guess pulse. In addition, we have to choose `lambda_a` for each control
field. This parameter controls the magnitude of the updates for the respective fields in each iterative step of the optimization algorithm. These options are collected in a dictionary of `pulse_options`:

In [ ]:
pulse_options = {H[1][1]: dict(lambda_a=25, update_shape=S)}

We can now collect everything into a call to `optimize_pulse`:

In [ ]:
oct_result = krotov.optimize_pulses(
    objectives,
    pulse_options=pulse_options,
    tlist=tlist,
    propagator=krotov.propagators.expm,
    chi_constructor=krotov.functionals.chis_ss,
    info_hook=krotov.info_hooks.print_table(
        J_T=krotov.functionals.J_T_ss,
        show_g_a_int_per_pulse=False,
        unicode=False,
    ),
    check_convergence=krotov.convergence.Or(
        krotov.convergence.value_below(1e-3, name="J_T"),
        krotov.convergence.check_monotonic_error,
    ),
    iter_stop=50,
)

## Simulate dynamics of the optimized field

Having obtained the optimized control field, we can now plot it and calculate
the population dynamics under this field.

In [ ]:
plt.close("all")
plot_pulse(oct_result.optimized_controls[0], tlist)

opt_dynamics = oct_result.optimized_objectives[0].mesolve(
    tlist, e_ops=[proj0, proj1]
)
plot_population(opt_dynamics)

opt_dynamics = oct_result.optimized_objectives[0].mesolve(
    tlist, e_ops=[σ_x, σ_y, σ_z]
)
plot_bloch(opt_dynamics)

## Further Tasks

1) Vary the numerical parameters `lambda_a` and $n_{t}$ to improve the
optimization. You should be able to reach the desired fidelity of 99% within
less than 50 iterations.

2) Try to improve the guess pulse to speed up the convergence. Hint: The interesting
parameters are `A` and $T$ resp. `t_stop` (Keep in mind to change it in the
shape $S$ and in the time grid `tlist`). Note that a constant pulse might not be
the best option as a guess pulse.

## Next steps

You are now ready to advance to a more sophisticated example for gradient-based optimization. For instance, [Exercise III.2](py_exercise_3_2_lambda.ipynb) covers the STIRAP protocol in the three-level lambda system. If you are more interested in gate optimization for quantum information applications, we recommend [Exercise III.4](py_exercise_3_4_gate.ipynb).

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_3_1_py)
<!-- Autofooter end -->